***GENERATED CODE FOR asthmaclassify PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DoctorInCharge', 'transformation_label': 'String Indexer'}], 'feature': 'DoctorInCharge', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Dr_Confid', 'max': 'Dr_Confid', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DoctorInCharge'}, {'feature_label': 'DoctorInCharge', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DoctorInCharge')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run asthmaclassifyHooks.ipynb
try:
	#sourcePreExecutionHook()

	asthmadiseasedata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/asthma_disease_data.csv', 'filename': 'asthma_disease_data.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(asthmadiseasedata)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run asthmaclassifyHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(asthmadiseasedata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5283.5", "stddev": "144.48", "min": "5034", "max": "5533", "missing": "0"}, "updatedLabel": "PatientID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "42.44", "stddev": "22.18", "min": "5", "max": "79", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Gender", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.47", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ethnicity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.73", "stddev": "1.01", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "Ethnicity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EducationLevel", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.37", "stddev": "0.88", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "EducationLevel"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BMI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "27.34", "stddev": "7.04", "min": "15.03180339", "max": "39.93958909", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BMI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Smoking", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.14", "stddev": "0.35", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Smoking"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PhysicalActivity", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "4.92", "stddev": "2.88", "min": "0.035913957", "max": "9.96027496", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PhysicalActivity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DietQuality", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "5.04", "stddev": "2.88", "min": "0.003030847", "max": "9.999903701", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DietQuality"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SleepQuality", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.13", "stddev": "1.74", "min": "4.004424776", "max": "9.995606902", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SleepQuality"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PollutionExposure", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "5.17", "stddev": "2.88", "min": "0.058103345", "max": "9.993622329", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PollutionExposure"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PollenExposure", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "5.34", "stddev": "2.93", "min": "7.36387E-4", "max": "9.999554769", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PollenExposure"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DustExposure", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "4.93", "stddev": "2.8", "min": "0.002433833", "max": "9.986299409", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DustExposure"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PetAllergy", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.17", "stddev": "0.38", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "PetAllergy"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FamilyHistoryAsthma", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.29", "stddev": "0.45", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FamilyHistoryAsthma"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "HistoryOfAllergies", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.38", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "HistoryOfAllergies"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eczema", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.2", "stddev": "0.4", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Eczema"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "HayFever", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.25", "stddev": "0.43", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "HayFever"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "GastroesophagealReflux", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.17", "stddev": "0.37", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "GastroesophagealReflux"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LungFunctionFEV1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.54", "stddev": "0.88", "min": "1.000458788", "max": "3.999718602", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LungFunctionFEV1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LungFunctionFVC", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.64", "stddev": "1.29", "min": "1.500044909", "max": "5.999420516", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LungFunctionFVC"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Wheezing", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.59", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Wheezing"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ShortnessOfBreath", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.52", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "ShortnessOfBreath"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ChestTightness", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.5", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "ChestTightness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Coughing", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.54", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Coughing"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NighttimeSymptoms", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.6", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "NighttimeSymptoms"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ExerciseInduced", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.59", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "ExerciseInduced"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Diagnosis", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Diagnosis"}, {"transformationsData": [{"feature_label": "DoctorInCharge", "transformation_label": "String Indexer"}], "feature": "DoctorInCharge", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Dr_Confid", "max": "Dr_Confid", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DoctorInCharge"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run asthmaclassifyHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["PatientID", "Age", "Gender", "Ethnicity", "EducationLevel", "BMI", "Smoking", "PhysicalActivity", "DietQuality", "SleepQuality", "PollutionExposure", "PollenExposure", "DustExposure", "PetAllergy", "FamilyHistoryAsthma", "HistoryOfAllergies", "Eczema", "HayFever", "GastroesophagealReflux", "LungFunctionFEV1", "LungFunctionFVC", "Wheezing", "ShortnessOfBreath", "ChestTightness", "Coughing", "NighttimeSymptoms", "ExerciseInduced", "DoctorInCharge_stringindexer"], "Diagnosis")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

